## Procesamiento de DataSets


### Procesamiento dataset n1


In [ ]:
import csv
from pathlib import Path
from unidecode import unidecode
from funciones import renombrar_prov

path_read= Path ('../datasets/ar-airports.csv')
path_write= Path ('../datasets_custom/ar-airports-custom.csv')
path_read2= Path ('../datasets/ar.csv')

Se hace el import de la libreria csv, de la libreria unidecode y de la funcion Path de la liberaria pathlib. Se declara en variables diferentes los paths a los archivos que utilizaremos.


In [ ]:
lectura = open (path_read, mode='r',encoding= 'UTF-8') 
lectura2 = open (path_read2, mode='r', encoding= 'UTF-8')
escritura= open(path_write ,mode='w',encoding= 'UTF-8' ,newline= '') 


Se abre los archivos de lectura en modo lectura ('r') y el archivo de escritura en modo escritura ('w'). También se especifica la codificación (UTF-8) y se establece newline=' ' para manejar los saltos de línea.

In [ ]:
lectorcsv= csv.reader(lectura)
lectorcsv2=csv.DictReader (lectura2)
escritorcsv= csv.writer (escritura)

Se crea un lector CSV (lectorcsv) para leer el archivo de entrada de los aeropuertos, un lector CSV de tipo dict (lectorcsv2) para leer el archivo de entrada de las provincias y ciudades de manera mas facil y un escritor CSV (escritorcsv) para escribir en el archivo de salida.

In [ ]:
encabezado = next(lectorcsv)
encabezado.append ('prov_name')
encabezado.append ('elevation_name')
escritorcsv.writerow (encabezado)

Se lee la primera fila del archivo de entrada para obtener el encabezado. Luego, se agregan las cadenas 'elevation_name' y 'prov_name' al encabezado. El encabezado actualizado se escribe en el archivo de salida.

In [ ]:

ciudad_prov= {}
for line in lectorcsv2:
    ciudad_prov [unidecode(line['city'])] = line ['admin_name']

ciudad_prov ['Buenos Aires'] = 'Ciudad Autónoma de Buenos Aires' # Buenos Aires se hace a mano porque esta mal escrito en el dataset.

Recorro el dataset ar.csv y guardo en un diccionario el nombre de las ciudades y su respectiva provincia.

In [ ]:
for fila in lectorcsv:
    city = unidecode(fila[13])
    aux= renombrar_prov.renombar_prov (fila[10]) #consultar la funcion "renombrar_prov"
    prov = ciudad_prov.get(city, aux)  
    if prov== 'Tierra del Fuego':
        prov= 'Tierra del Fuego, Antártida e Islas del Atlántico Sur' # No encontramos forma mas eficiente de hacer este cambio para que las consultas 
    # de los datasets funcionen correctamente. El cambio del nombre de Tierra del Fuego para que sea homogeneo en todo el programa es necesaria para el 
    # correcto funcionamiento de las demas consultas. 
    fila.append(prov)

    try: 
        elevacion = float(fila[6])
        if elevacion <= 131:
            elevation_name= 'bajo'
        elif elevacion <=903:
            elevation_name= 'medio'
        else:   
            elevation_name='alto'

        fila.append(elevation_name)
        escritorcsv.writerow(fila)

    except ValueError: 
        fila.append ('Sin altura')
        escritorcsv.writerow(fila)

Para cada fila restante en el archivo de entrada se agrega el nombre de la provincia a la linea utilizando el diccionario previamente creado (se utiliza unidecode para evitar errores por tildes), dado que en el dataset ar.csv NO estan todas las ciudades de las provincias, si sucede que la ciudad del aeropuerto no esta en el diccionario, utilizo la provincia del mismo dataset de los aeropuertos en la columna 10 (invocando la funcion renombrar_prov que elimina la palabra 'province'). Posteriormente, se intenta convertir el valor en la sexta columna (la cual es la elevacion del aeropuerto) a un número flotante. Si la conversión es exitosa, se procede a determinar la categoría de elevación ('bajo', 'medio' o 'alto') y se agrega al archivo csv en la columna creada previamente. Si la conversión falla (porque la linea esta vacia o no es un numero), se captura la excepción ValueError y se agrega la cadena 'Sin altura' a la fila.


In [ ]:
lectura.close()
lectura2.close()
escritura.close()

Cierre de archivos.

---

### Procesamiento dataset n2

Creo las rutas de los archivos que seran el origen de los datos conectivida y la carpeta en que se creara el nuevo archivo con los datos procesados

In [3]:
from pathlib import Path

conectividad = Path('..','datasets') / "Conectividad_Internet.csv"
conectividad_modificado = Path('..','datasets_custom') / "Conectividad_Internet.csv"

Haciendo uso de la funcion "create_modified_file()" de modules/Conexiones_internet, se genera un nuevo archivo con una nueva columna "posee_conectividad" si todos los campos que especifican los distintos tipos son "NO" la misma tendra "NO", con que uno solo de los tipos sea "SI" la columna tomara el valor "SI"

In [ ]:
from funciones import Conectividad_internet as connectivity
connectivity.create_modified_file(conectividad,conectividad_modificado)

---

### Procesamiento dataset n3

Primero se importan los módulos y la función con la que se va a trabajar. También se definen las constantes que indican los índices:

In [ ]:
import csv
import pathlib

from funciones.lagos_argentina import degrees_to_decimal

SUPERFICIE = 2
COORDENADAS = 5

Utilizando PathLib se crean las rutas del CSV de lectura y del CSV de escritura:

In [ ]:
read_dataset = pathlib.Path('../datasets/lagos_arg.csv')
write_dataset = pathlib.Path('../datasets_custom/lagos_arg_custom.csv')

Se abren ambos archivos, uno en modo lectura, otro en modo escritura. Se indica la codificación **UTF-8**. Se usa *newline=''* para evitar líneas vacías entre las filas.

In [ ]:
read_file = open(read_dataset, mode='r', encoding='UTF-8', newline='')
write_file = open(write_dataset, mode='w', encoding='UTF-8', newline='')

También se crean los iterables **reader** y **writer** con los que se va a trabajar sobre los archivos: 

In [ ]:
reader = csv.reader(read_file)
writer = csv.writer(write_file)

Se toma el *header* del archivo de lectura y se elimina de ésta la columna **COORDENADAS**, ya que va a estar dividida en dos:

In [ ]:
reader_header = next(reader)
reader_header.pop (COORDENADAS);

Acto seguido se crea una lista en la que cada elemento es el texto de cada columna nueva. Luego se escribe el *header* del nuevo CSV concatenando la lista del header del archivo original y la lista de los nuevos elementos:

In [ ]:
extras_header = ['Sup Tamaño', 'Latitud(GMS)', 'Longitud(GMS)', 'Latitud(GD)', 'Longitud(GD)']
writer.writerow(reader_header + extras_header);

#### Procesamiento

Con un **For** sobre el iterable se realiza el procesamiento del archivo linea por linea.

**-Superficie:** En la variable *sup* se almacena el contenido de la columna *SUPERFICIE* del archivo. Al ser CSV el dato es leído como string, por esto mismo es que se realiza la conversión a entero con *int()*.

Utilizando dicha variable se compara si el valor es chico/medio/grande y se agrega al final de la linea, en la columna de *Sup Tamaño*.


**-Coordenadas:** El String de la columna **COORDENADAS** se divide utilizando *split* (con el espacio entre medio), y ambas partes se guardan en *coordinates*. 

Acto seguido se elimina el String de la columna **COORDENADAS** y se agregan al final los valores separados que están en la lista *coordinates*.

Finalmente empleando la misma lista *coordinates*: Se agregan los valores al final de la línea pero en este caso llamando a la función *degrees_to_decimal*. Esta función toma como argumento el String de coordinadas en formato GMS(grados, minutos y segundos) y lo retorna en GD(grados decimales).

In [ ]:
for line in reader:
    # Superficie
    sup = int(line[SUPERFICIE])

    if sup <= 17:
        line.append ('chico')
    elif sup <= 59:
        line.append ('medio')
    else:
        line.append ('grande')

    # Coordenadas
    coordinates = line[COORDENADAS].split(' ')
    line.pop(COORDENADAS)

    line.append (coordinates[0]) 
    line.append (coordinates[1]) 

    line.append (degrees_to_decimal(coordinates[0]))
    line.append (degrees_to_decimal(coordinates[1])) 

    writer.writerow(line)

Para terminar se cierran ambos archivos:

In [ ]:
read_file.close()
write_file.close()

---

### Procesamiento dataset n4

Importo el módulo "censo2022" desde la carpeta "funciones" e invoco a la función "create_modified_file" para generar un nuevo archivo .csv en el directorio "datasets_custom" a partir de los datos existentes en el archivo "c2022_tp_c_resumen_adaptado.csv".
Si ya existe el archivo modificado, lo sobreescribe.

In [4]:
from funciones import censo2022 as cd
from pathlib import Path
file_route = Path('..','datasets')/ 'c2022_tp_c_resumen_adaptado.csv'
new_file_route = Path('..','datasets_custom')/ 'Censo_Modificado.csv'
cd.create_modified_file(file_route,new_file_route)